In [55]:
import pandas as pd

In [56]:
import mysql.connector as msql
from mysql.connector import Error
dataset = []
try:
    conn = msql.connect(host='localhost', database='DMHW1', user='root', password='TP6XU4JO3@@')
    if conn.is_connected():
        mycursor = conn.cursor()
        mycursor.execute("select database();")
        record = mycursor.fetchone()
        print("You're connected to database: ", record)
        mycursor.execute("""
                            select 
                                pc.product_category,
                                #c.product_name, 
                                d.gender,
                                CONCAT('totally ',d.total_children,' children') as 'total_children',
                                CONCAT('own ', d.num_cars_owned ,' cars') as 'owned_cars',
                                CONCAT('totally ', d.num_children_at_home,' children at home') as 'children_at_home',
                                CASE WHEN d.houseowner = 'Y' then 'Is houseowner'
                                else 'Is not houseowner'
                                end as 'houseowner',
                                CASE WHEN b.day_of_month between 1 and 10 then 'month_begin'
                                WHEN b.day_of_month between 11 and 20 then 'month_middle'
                                ELSE 'month_end'
                                END as 'day_of_the_month'
                                #pr.promotion_name
                                from DMHW1.sales_fact_1998 a 
                                join DMHW1.time_by_day b
                                on a.time_id = b.time_id
                                join DMHW1.product c 
                                on a.product_id = c.product_id 
                                join DMHW1.product_class pc
                                on c.product_class_id = pc.product_class_id
                                join DMHW1.customer d 
                                on a.customer_id = d.customer_id 
                                join DMHW1.promotion pr
                                on a.promotion_id = pr.promotion_id
                                where b.the_month != 'December'
                                union 
                                select 
                                pc.product_category,
                                #c.product_name,
                                d.gender, 
                                CONCAT('totally ',d.total_children,' children') as 'total_children',
                                CONCAT('own ', d.num_cars_owned ,' cars') as 'owned_cars',
                                CONCAT('totally ', d.num_children_at_home,' children at home') as 'children_at_home',
                                CASE WHEN d.houseowner = 'Y' then 'Is houseowner'
                                else 'Is not houseowner'
                                end as 'houseowner',
                                CASE WHEN b.day_of_month between 1 and 10 then 'month_begin'
                                WHEN b.day_of_month between 11 and 20 then 'month_middle'
                                ELSE 'month_end'
                                END as 'day_of_the_month'
                                #pr.promotion_name
                                from DMHW1.sales_fact_dec_1998 a
                                join DMHW1.time_by_day b
                                on a.time_id = b.time_id
                                join DMHW1.product c 
                                on a.product_id = c.product_id 
                                join DMHW1.product_class pc
                                on c.product_class_id = pc.product_class_id
                                join DMHW1.customer d 
                                on a.customer_id = d.customer_id 
                                join DMHW1.promotion pr
                                on a.promotion_id = pr.promotion_id
                                where b.the_month != 'December' 
                        """)
        print("data is fetched....")
        #loop through the data fetched
        results = mycursor.fetchall()
        a = dict()
        for item in results:
            lis = []
            row = list(item)
            for i in range(len(row)):
                if type(row[i]) is not str:
                    row[i] = str(row[i])
                lis.append(row[i])
            #print(lis)
            dataset.append(lis)
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('dmhw1',)
data is fetched....


In [57]:
from mlxtend.frequent_patterns import fpmax, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [58]:
frequent_sets = fpgrowth(df, min_support=0.02,use_colnames=True) 

In [59]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_sets, metric='confidence', min_threshold=0.8)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x:len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x:len(x))
#rules[(rules["antecedent_len"] >= 2)& (rules["consequent_len"] >=2)].sort_values(by=['support','confidence'], ascending=False).to_csv('has_promotion.csv')
rules[(rules["antecedent_len"] >= 1)& (rules["consequent_len"] >=1)].sort_values(by=['confidence'], ascending=False).to_csv('not_chrismas_mining_3.csv')
